In [ ]:
pip install ccxt pandas

In [ ]:
import ccxt
import pandas as pd

# Initialize the CCXT exchange object
exchange = ccxt.binance()

# Fetch all available trading pairs on Binance
symbols = exchange.fetch_markets()
symbol_names = [symbol['symbol'] for symbol in symbols]

# Filter symbols based on USDT or USD
filtered_symbols = [symbol for symbol in symbol_names if 'USDT' in symbol or 'USD' in symbol]

# Define the timeframe for OHLCV (Open, High, Low, Close, Volume) data to daily (1 day)
timeframe = '1d'

def fetch_ohlcv(symbol, timeframe, limit=100):
    # Fetch historical OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    
    # Convert the data to a Pandas DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    return df


def detect_engulfing_pattern(df):
    """
    Detect bullish and bearish engulfing patterns in a DataFrame of OHLCV data.

    Parameters:
    - df: Pandas DataFrame containing OHLCV data (columns: timestamp, open, high, low, close, volume)

    Returns:
    - engulfing_patterns: DataFrame containing rows with detected engulfing patterns
    """
    engulfing_patterns = pd.DataFrame(columns=['timestamp', 'pattern'])

    for i in range(1, len(df)):
        current_candle = df.iloc[i]
        previous_candle = df.iloc[i - 1]

        # Bullish Engulfing: Current candle opens below the previous close and closes above the previous open
        bullish_engulfing = current_candle['open'] < previous_candle['close'] and current_candle['close'] > previous_candle['open']

        # Bearish Engulfing: Current candle opens above the previous close and closes below the previous open
        bearish_engulfing = current_candle['open'] > previous_candle['close'] and current_candle['close'] < previous_candle['open']

        if bullish_engulfing:
            engulfing_patterns = engulfing_patterns.append({'timestamp': current_candle['timestamp'], 'pattern': 'Bullish Engulfing'}, ignore_index=True)
        elif bearish_engulfing:
            engulfing_patterns = engulfing_patterns.append({'timestamp': current_candle['timestamp'], 'pattern': 'Bearish Engulfing'}, ignore_index=True)

    return engulfing_patterns
def detect_pin_bar(df, sensitivity = 0.2):
    """
    Detect pin bar patterns in a DataFrame of OHLCV data.

    Parameters:
    - df: Pandas DataFrame containing OHLCV data (columns: timestamp, open, high, low, close, volume)

    Returns:
    - pin_bar_patterns: DataFrame containing rows with detected pin bar patterns
    """
    pin_bar_patterns = pd.DataFrame(columns=['timestamp', 'pattern'])

    for i in range(1, len(df)):
        current_candle = df.iloc[i]
        previous_candle = df.iloc[i - 1]

        # Pin Bar: Small body relative to the length of its wick
        is_pin_bar = (
            (current_candle['high'] > max(previous_candle['open'], previous_candle['close'])) and
            (current_candle['low'] < min(previous_candle['open'], previous_candle['close'])) and
            (abs(current_candle['open'] - current_candle['close']) < sensitivity * abs(current_candle['high'] - current_candle['low']))
        )

        if is_pin_bar:
            pin_bar_patterns = pin_bar_patterns.append({'timestamp': current_candle['timestamp'], 'pattern': 'Pin Bar'}, ignore_index=True)

    return pin_bar_patterns
# Main script
for symbol in filtered_symbols:
    try:
        # Fetch historical data
        df = fetch_ohlcv(symbol, timeframe, limit = 20)
         
        # Detect engulfing patterns
        ##engulfing_patterns = detect_engulfing_pattern(df)
        
        # Detect pin bar patterns
        pin_bar_patterns = detect_pin_bar(df, sensitivity = 0.25)
        '''
        if not engulfing_patterns.empty:
            print(f"Engulfing Patterns for {symbol}:")
            print(engulfing_patterns)
            print("\n" + "="*40 + "\n")
        '''
        if not pin_bar_patterns.empty:
            print(f"Pin Bar Patterns for {symbol}:")
            print(pin_bar_patterns)
            print("\n" + "="*40 + "\n")
    except ccxt.NetworkError as e:
        print(f"Network error for {symbol}: {e}")
    except ccxt.ExchangeError as e:
        print(f"Exchange error for {symbol}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred for {symbol}: {e}")
